In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer

# 1. Datensatz laden
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')


# y_train und y_test in Series umwandeln
y_train = pd.read_csv('y_train.csv').iloc[:, 0]
y_test = pd.read_csv('y_test.csv').iloc[:, 0]

# 2. Angepasste RMSPE-Funktion
def rmspe(y_true, y_pred):
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# 3. Numerische und kategorische Features definieren
numerical_features = ['year', 'month', 'day', 'week_of_year', 'lag_1', 'lag_7']
already_encoded_features = ['Open', 'Promo', 'promo2']
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday', 'Assortment']

# 4. Preprocessor erstellen
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features + already_encoded_features),  
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)  
    ])

# 5. Pipeline erstellen
pipeline_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# 6. Pipeline mit den Trainingsdaten trainieren
pipeline_lr.fit(X_train, y_train)

# 7. Cross-Validation mit 5 Folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(pipeline_lr, X_train, y_train, cv=kf, scoring=rmspe_scorer, verbose=True)

# Durchschnittliche RMSPE aus Cross-Validation
mean_rmspe = np.mean(cv_scores)
print(f'Mean RMSPE (Cross-Validation): {mean_rmspe}')

# 8. Vorhersagen auf dem Testdatensatz machen
y_pred = pipeline_lr.predict(X_test)

# RMSPE auf Testdatensatz berechnen
test_rmspe = rmspe(y_test, y_pred)
print(f'RMSPE on Test Data: {test_rmspe}')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer
from prophet import Prophet

# Angepasste RMSPE-Funktion, die Tage mit 0 Sales ignoriert
def rmspe(y_true, y_pred):
    # Nur Fälle berücksichtigen, bei denen y_true nicht 0 ist
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Sicherstellen, dass das Datumsformat korrekt ist
data['Date'] = pd.to_datetime(data['Date'])

# Umwandeln der Daten in das Prophet-kompatible Format
df_prophet = data[['Date', 'Sales']].rename(columns={'Date': 'ds', 'Sales': 'y'})

# Aufteilen der Daten in Trainings- und Testsets (Beispiel: Daten vor Juli 2015 zum Trainieren)
train_data_prophet = df_prophet[df_prophet['ds'] < '2015-07-01']
test_data_prophet = df_prophet[df_prophet['ds'] >= '2015-07-01']

# Erstellen eines Prophet-Modells
model = Prophet()

# Trainieren des Prophet-Modells mit den Trainingsdaten
model.fit(train_data_prophet)

# Erstellen eines DataFrames für zukünftige Datenpunkte (Vorhersagen für die nächsten Testtage)
future = model.make_future_dataframe(periods=len(test_data_prophet), freq='D')

# Vorhersagen erstellen
forecast = model.predict(future)

# Vorhersagen anzeigen (yhat ist die vorhergesagte Sales-Wert)
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

# Zusammenführen der Vorhersagen mit den tatsächlichen Werten
forecast_with_actual = forecast[['ds', 'yhat']].merge(test_data_prophet, on='ds')

# RMSPE-Bewertung berechnen
test_rmspe_prophet = rmspe(forecast_with_actual['y'], forecast_with_actual['yhat'])
print("Test RMSPE for Prophet:", test_rmspe_prophet)

# Vorhersage-Plot
model.plot(forecast)

# Komponenten-Plot (Trend, wöchentliche und jährliche Saisonalität)
model.plot_components(forecast)


## 1. Versuch mit Timestamp Cross Validation und ohne lag feature n_neighbors = 2

In [24]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer  # make_scorer importieren
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Datensatz sortieren, falls nicht bereits geschehen (angenommen, du hast eine Spalte 'Date')
data = data.sort_values('Date')

# auch noch Text hinzufügen
data = data[data ['Open']!=0]
data = data[data ['Sales']>0]

# Zielvariable und Features definieren
X = data.drop(['Sales', 'Date', 'Open'], axis=1)  # 'Date' wird entfernt, wenn es keine erklärende Variable ist
y = data['Sales']

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year']
# Bereits encodierte Features
already_encoded_features = ['Open', 'Promo', 'promo2']
# Noch nicht encodierte kategorische Features
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday','Assortment']

df = pd.DataFrame(data)
# Bestimme die Grenze für 80% Training und 20% Test
train_size = int(len(df) * 0.8)
# Aufteilen in Trainings- und Test-Datensätze
train = df.iloc[:train_size]
test = df.iloc[train_size:]
# Ziel- und Eingabedaten für Training und Test
X_train = train.drop('Sales', axis=1)
y_train = train['Sales']
X_test = test.drop('Sales', axis=1)
y_test = test['Sales']

# Angepasste RMSPE-Funktion, die Tage mit 0 Sales ignoriert
def rmspe(y_true, y_pred):
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))
# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Erstelle den Preprocessor für numerische und kategorische Features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features + already_encoded_features),
        ('enc', 'passthrough', already_encoded_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)
    ])
# Erstelle die Pipeline für KNN
pipeline_knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=2)) # Hyperparameter zum anpassen hoch und runter
])


# Trainiere das KNN-Modell
pipeline_knn.fit(X_train, y_train)
# 5-fold TimeSeries Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)
# Cross-Validation mit TimeSeriesSplit
cv_scores_knn = cross_val_score(pipeline_knn, X_train, y_train, cv=tscv, scoring=rmspe_scorer, verbose=True)

# Durchschnittlichen RMSPE berechnen
mean_rmspe_knn = np.mean(cv_scores_knn)
print("Durchschnittlicher RMSPE (KNN, TimeSeries Cross-Validation):", mean_rmspe_knn)
# 3. Vorhersagen auf dem Testset machen
y_pred_knn = pipeline_knn.predict(X_test)
# Test RMSPE berechnen
test_rmspe_knn = rmspe(y_test, y_pred_knn)
print("Test RMSPE (KNN):", test_rmspe_knn)


Durchschnittlicher RMSPE (KNN, TimeSeries Cross-Validation): -0.30225682404536586
Test RMSPE (KNN): 0.24405497742782506


## 1. Durchschnittlicher RMSPE ist schlechter geworden (von -0.184 auf -0.232)
**Was es bedeutet**:
- Der RMSPE-Wert in der zeitbasierten Cross-Validation ist höher geworden. Das bedeutet, dass das Modell auf den Trainingsdaten in der zeitlichen Cross-Validation schlechter abschneidet als zuvor in der regulären Cross-Validation.

**Gründe für die Verschlechterung:**
- Zeitabhängigkeit der Daten: Die Daten, die du modellierst, haben eine zeitliche Komponente, und das Modell hat Schwierigkeiten, die Muster über die Zeit hinweg zu verallgemeinern. Die nicht-zeitliche KFold-Cross-Validation vermischt die Zeiträume, was möglicherweise eine unrealistisch bessere Leistung ergab. Jetzt, wo die Cross-Validation die Reihenfolge respektiert, zeigt sich, dass das Modell nicht so gut verallgemeinert.
**Trainingsmengen:** 
- Bei zeitbasierter Cross-Validation wird das Modell in den ersten Folds mit einer kleineren Datenmenge trainiert (weil frühere Zeitpunkte für das Training und spätere für das Testen verwendet werden). Das kann zu schlechteren Ergebnissen führen, da das Modell mit weniger Daten trainiert wird.


## 2. Vergleich mit dem Test RMSPE (0.253)

- Der RMSPE aus der zeitbasierten Cross-Validation (0.232) liegt immer noch unter dem RMSPE auf dem Testset (0.253), was bedeutet, dass das Modell immer noch tendenziell schlechter auf den Testdaten abschneidet, aber die Werte sind näher beieinander.

**Was das bedeutet:** 
- Die zeitbasierte Cross-Validation scheint jetzt eine realistischere Abschätzung der Modellleistung zu liefern, da die Abweichung zwischen dem RMSPE aus der Cross-Validation und dem Test-RMSPE kleiner geworden ist. Dies deutet darauf hin, dass die nicht-zeitliche Cross-Validation das Modell möglicherweise überoptimistisch bewertet hat. (Overfitting?)


## Fazit:
- Die Verschlechterung des RMSPE nach Einführung der zeitbasierten Cross-Validation ist ein Hinweis darauf, dass das Modell Schwierigkeiten hat, Muster in zeitlich sortierten Daten zu lernen.
- Der Unterschied zwischen der Cross-Validation-Leistung (0.232) und der Testleistung (0.253) ist jedoch kleiner geworden, was bedeutet, dass die zeitbasierte Cross-Validation eine realistischere Abschätzung der Modellleistung bietet.
- Möglicherweise musst du das Modell weiter verbessern, insbesondere um die Zeitabhängigkeit der Daten besser zu erfassen, z. B. durch Feature-Engineering (Hinzufügen von Zeittrends oder saisonalen Mustern) oder durch den Einsatz von Modellen, die sich besser für zeitliche Daten eignen, wie z. B. ARIMA, Prophet, oder Recurrent Neural Networks (RNNs).

## 2. Versuch Hyperparametertuning (n_neighbors = 6) und Feature Engineering (testen von versch. features aus kaggle) zur Verbesserung des Modells

In [25]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
import pandas as pd
import numpy as np

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)
# Datensatz sortieren, falls nicht bereits geschehen (angenommen, du hast eine Spalte 'Date')
data = data.sort_values('Date')

# auch noch Text hinzufügen
data = data[data ['Open']!=0]
data = data[data ['Sales']>0]

# Zielvariable und Features definieren
X = data.drop(['Sales', 'Date', 'Open'], axis=1)  # 'Date' wird entfernt, wenn es keine erklärende Variable ist
y = data['Sales']
# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year']
# Bereits encodierte Features
already_encoded_features = ['Open', 'Promo', 'promo2']
# Noch nicht encodierte kategorische Features
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday','Assortment']

# 1. Datensatz aufteilen in Trainings- und Testdaten
df = pd.DataFrame(data)

# Bestimme die Grenze für 80% Training und 20% Test
train_size = int(len(df) * 0.8)
# Aufteilen in Trainings- und Test-Datensätze
train = df.iloc[:train_size]
test = df.iloc[train_size:]
# Ziel- und Eingabedaten für Training und Test
X_train = train.drop('Sales', axis=1)
y_train = train['Sales']
X_test = test.drop('Sales', axis=1)
y_test = test['Sales']

# Angepasste RMSPE-Funktion, die Tage mit 0 Sales ignoriert
def rmspe(y_true, y_pred):
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))
# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Erstelle den Preprocessor für numerische und kategorische Features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features + already_encoded_features),
        ('enc', 'passthrough', already_encoded_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)
    ])
# Erstelle die Pipeline für KNN
pipeline_knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=6))  #hier n_neighbors anpassen
])

# Hyperparameter-Tuning mit GridSearchCV
param_grid = {
    'model__n_neighbors': [3, 5, 10, 15],   # Verschiedene Werte für k
    'model__weights': ['uniform', 'distance'],  # Gewichtete oder uniforme Nachbarn
    'model__metric': ['euclidean', 'manhattan']  # Verschiedene Distanzmetriken
}
# Trainiere das KNN-Modell
pipeline_knn.fit(X_train, y_train)
# 5-fold TimeSeries Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)
# Cross-Validation mit TimeSeriesSplit
cv_scores_knn = cross_val_score(pipeline_knn, X_train, y_train, cv=tscv, scoring=rmspe_scorer, verbose=True)

# Durchschnittlichen RMSPE berechnen
mean_rmspe_knn = np.mean(cv_scores_knn)
print("Durchschnittlicher RMSPE (KNN, TimeSeries Cross-Validation):", mean_rmspe_knn)
# 3. Vorhersagen auf dem Testset machen
y_pred_knn = pipeline_knn.predict(X_test)
# Test RMSPE berechnen
test_rmspe_knn = rmspe(y_test, y_pred_knn)
print("Test RMSPE (KNN):", test_rmspe_knn)


Durchschnittlicher RMSPE (KNN, TimeSeries Cross-Validation): -0.2794781299440422
Test RMSPE (KNN): 0.22253414113163228


## Versuch mit Feature wie CompetitionOpen und Promo2Open

In [2]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
import pandas as pd
import numpy as np

In [26]:
# Laden des Datensatzes
data_cleaned = "../data/cleaned_train.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)
# Datensatz sortieren, falls nicht bereits geschehen (angenommen, du hast eine Spalte 'Date')
data = data.sort_values('Date')
# auch noch Text hinzufügen
data = data[data ['Open']!=0]
data = data[data ['Sales']>0]
# Zielvariable und Features definieren
X = data.drop(['Sales', 'Date', 'Open'], axis=1)  # 'Date' wird entfernt, wenn es keine erklärende Variable ist
y = data['Sales']
# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year']
# Bereits encodierte Features
already_encoded_features = ['Open', 'Promo', 'promo2']
# Noch nicht encodierte kategorische Features
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday','Assortment']
# 1. Datensatz aufteilen in Trainings- und Testdaten
df = pd.DataFrame(data)

## Hier n_neighbors anpassen und GridSearch/Randomized Search/ Optuna definieren

In [32]:
import optuna
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
import pandas as pd
import numpy as np

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)
# Datensatz sortieren, falls nicht bereits geschehen (angenommen, du hast eine Spalte 'Date')
data = data.sort_values('Date')

# auch noch Text hinzufügen
data = data[data ['Open']!=0]
data = data[data ['Sales']>0]

# Zielvariable und Features definieren
X = data.drop(['Sales', 'Date', 'Open'], axis=1)  # 'Date' wird entfernt, wenn es keine erklärende Variable ist
y = data['Sales']
# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year']
# Bereits encodierte Features
already_encoded_features = ['Open', 'Promo', 'promo2']
# Noch nicht encodierte kategorische Features
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday','Assortment']

# 1. Datensatz aufteilen in Trainings- und Testdaten
df = pd.DataFrame(data)

# Bestimme die Grenze für 80% Training und 20% Test
train_size = int(len(df) * 0.8)
# Aufteilen in Trainings- und Test-Datensätze
train = df.iloc[:train_size]
test = df.iloc[train_size:]
# Ziel- und Eingabedaten für Training und Test
X_train = train.drop('Sales', axis=1)
y_train = train['Sales']
X_test = test.drop('Sales', axis=1)
y_test = test['Sales']

# Angepasste RMSPE-Funktion, die Tage mit 0 Sales ignoriert
def rmspe(y_true, y_pred):
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))
# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Erstelle den Preprocessor für numerische und kategorische Features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features + already_encoded_features),
        ('enc', 'passthrough', already_encoded_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)
    ])
# Erstelle die Pipeline für KNN
pipeline_knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=6))  #hier n_neighbors anpassen
])

# Hyperparameter-Tuning mit GridSearchCV / RandomizedCV / Optuna
param_grid = {
    'model__n_neighbors': [3, 5, 10, 15],   # Verschiedene Werte für k
    'model__weights': ['uniform', 'distance'],  # Gewichtete oder uniforme Nachbarn
    'model__metric': ['euclidean', 'manhattan']  # Verschiedene Distanzmetriken
}
# Trainiere das KNN-Modell
pipeline_knn.fit(X_train, y_train)
# 5-fold TimeSeries Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)
# Cross-Validation mit TimeSeriesSplit
cv_scores_knn = cross_val_score(pipeline_knn, X_train, y_train, cv=tscv, scoring=rmspe_scorer, verbose=True)


# Definiere die Optuna-Ziel-Funktion
def objective(trial):
    # Hyperparameter, die Optuna optimieren soll
    n_neighbors = trial.suggest_int('n_neighbors', 2, 20)  # Optimierung von n_neighbors zwischen 2 und 20
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])  # 'uniform' oder 'distance'
    p = trial.suggest_int('p', 1, 2)  # Metrik: 1 = Manhattan, 2 = Euklidische Distanz
    # KNN Modell mit den von Optuna vorgeschlagenen Parametern
    model = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p) 
    # Pipeline, die den Preprocessor und das KNN-Modell enthält
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])   
    # Cross-Validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=tscv, scoring=rmspe_scorer, n_jobs=-1)
    # Durchschnittlicher RMSPE (negative Werte, weil 'greater_is_better=False')
    mean_rmspe = np.mean(cv_scores)
    return mean_rmspe  # Minimierung des RMSPE
# Optuna-Studie starten
study = optuna.create_study(direction='minimize')  # Wir wollen den Fehler minimieren
study.optimize(objective, n_trials=50)  # Probiere 50 verschiedene Hyperparameter-Kombinationen aus
# Beste Parameter anzeigen
print("Beste Parameter: ", study.best_params)
# Trainiere das Modell mit den besten Parametern
best_n_neighbors = study.best_params['n_neighbors']
best_weights = study.best_params['weights']
best_p = study.best_params['p']
# Bestes Modell trainieren
best_model = KNeighborsRegressor(n_neighbors=best_n_neighbors, weights=best_weights, p=best_p)
pipeline_best = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', best_model)
])
pipeline_best.fit(X_train, y_train)
# Vorhersagen auf dem Testset
y_pred_best = pipeline_best.predict(X_test)
test_rmspe_best = rmspe(y_test, y_pred_best)

print("Test RMSPE (bestes KNN-Modell):", test_rmspe_best)



# Durchschnittlichen RMSPE berechnen
mean_rmspe_knn = np.mean(cv_scores_knn)
print("Durchschnittlicher RMSPE (KNN, TimeSeries Cross-Validation):", mean_rmspe_knn)
# 3. Vorhersagen auf dem Testset machen
y_pred_knn = pipeline_knn.predict(X_test)
# Test RMSPE berechnen
test_rmspe_knn = rmspe(y_test, y_pred_knn)
print("Test RMSPE (KNN):", test_rmspe_knn)

C:\Users\iTTaste\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
